In [1]:
!git clone https://github.com/WaShindeiru/IceCubeNeutrino

Cloning into 'IceCubeNeutrino'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 95 (delta 48), reused 33 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 798.65 KiB | 13.31 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
!cp IceCubeNeutrino/* .

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip ./drive/MyDrive/icecube/ice_cube_small.zip

Archive:  ./drive/MyDrive/icecube/ice_cube_small.zip
   creating: ice_cube_small/
  inflating: ice_cube_small/train_meta.parquet  
  inflating: ice_cube_small/test_meta.parquet  
  inflating: ice_cube_small/sensor_geometry.csv  
  inflating: ice_cube_small/sample_submission.parquet  
   creating: ice_cube_small/train_2/
  inflating: ice_cube_small/train_2/batch_3.parquet  
  inflating: ice_cube_small/train_2/batch_2.parquet  
  inflating: ice_cube_small/train_2/batch_1.parquet  
   creating: ice_cube_small/test/
  inflating: ice_cube_small/test/batch_661.parquet  


In [5]:
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from Resnet import resnet34

In [6]:
sensor = pd.read_csv('./ice_cube_small/sensor_geometry.csv')
train_meta = pq.ParquetFile('./ice_cube_small/train_meta.parquet')
it = train_meta.iter_batches()
train_meta = next(it).to_pandas()

In [7]:
train_meta.head(20)

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498
1,1,41,61,111,0.417742,1.549686
2,1,59,112,147,1.160466,2.401942
3,1,67,148,289,5.845952,0.759054
4,1,72,290,351,0.653719,0.939117
5,1,77,352,401,0.011372,1.295371
6,1,79,402,717,3.533397,2.479947
7,1,82,718,762,5.252108,0.672366
8,1,121,763,803,3.084929,2.065880
9,1,127,804,846,6.154335,1.371721


In [8]:
sensor.head(20)

,sensor_id,x,y,z
0,0,-256.14,-521.08,496.03
1,1,-256.14,-521.08,479.01
2,2,-256.14,-521.08,461.99
3,3,-256.14,-521.08,444.97
4,4,-256.14,-521.08,427.95
5,5,-256.14,-521.08,410.93
6,6,-256.14,-521.08,393.91
7,7,-256.14,-521.08,376.88
8,8,-256.14,-521.08,359.86
9,9,-256.14,-521.08,342.84


In [9]:
import torch.nn as nn
model = resnet34(pretrained = False)
model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=512, out_features=2))

In [10]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float).to(device)
    actual = torch.tensor([], dtype=torch.float).to(device)
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, values = data['input_tensor'].float().to(device),data['label'].to(device)
            outputs = model(inputs).to(device)
            predictions = torch.cat((predictions, torch.stack([torch.argmax(o) for o in outputs])),0)
            actual = torch.cat((actual, values), 0)
    predictions = predictions.cpu().numpy()
    actual = actual.cpu().numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    return rmse

In [14]:
from torch import optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'


pqfile = './ice_cube_small/train_meta.parquet'
path_batch = './ice_cube_small/train_2/'
geometry = "./ice_cube_small/sensor_geometry.csv"

batch_num=1
val_batch_num=2

# lr = 1e-06
lr = 1e-07
num_epochs = 1
batch_size = 4

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss_function = nn.MSELoss().to(device)
loss_function = nn.L1Loss(reduction='mean').to(device)

In [15]:
from torch.utils.data import DataLoader
from Icecube_Dataloader import IceCube_Dataloader, collate_fn

ice_dataset = IceCube_Dataloader(pqfile, path_batch, geometry, batch_num)
train_dataset = ice_dataset

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=11)

val_dataset = IceCube_Dataloader(pqfile, path_batch, geometry, val_batch_num)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=11)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [16]:
ice_dataset.__getitem__(0)['input_tensor'].shape

torch.Size([5, 1, 13])

In [17]:
print(device)

cuda


In [18]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    'val_dataloader': val_dataloader,
    'device':device,
    'num_epoch' : num_epochs
}


In [19]:
import gc
train_losses = []
val_losses = []
outputs_ = []
accum_iter = 1

def train(model, params):
    model.train()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    val_dataloader=params["val_dataloader"]
    val_iter = iter(val_dataloader)

    aa = 0

    device=params["device"]
    for epoch in range(0, num_epochs):
        for dat in tqdm(train_dataloader):
            # train step
            model.train()
            inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)

            assert not torch.isnan(inputs).any(), "Inputs contain NaNs"
            assert not torch.isnan(labels).any(), "Labels contain NaNs"

            outputs = model(inputs).to(device)
            outputs_.append(outputs.cpu().detach().numpy())

            train_loss = loss_function(outputs.float(),labels.float())
            train_losses.append(train_loss.item())


            # if aa%100==0:
            #     print(f"Train loss: {train_loss.item()}")

            train_loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()

            # validation step
            model.eval()

            with torch.no_grad():
                try:
                    temp = next(val_iter)
                    val_input = temp['input_tensor'].to(device)
                    val_label = temp['label'].to(device)
                    val_outputs = model(val_input).to(device)
                    val_loss = loss_function(val_outputs.float(),val_label.float())
                    val_losses.append(val_loss.item())
                except StopIteration:
                    pass

            aa = aa+1

    model.eval()
    gc.collect()

    torch.cuda.empty_cache()
    return 0


train(model, params)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 11 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  0%|          | 0/50000 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


plt.figure()
plt.plot(train_losses)

In [ ]:
plt.figure()
plt.plot(val_losses)

In [ ]:
from datetime import datetime

now = datetime.now()

timestamp = now.strftime("%Y-%m-%d_%H:%M:%S")

In [ ]:
name = "./weights/model_34_"+timestamp+".pth"
torch.save(model.state_dict(), name)

In [ ]:
import pickle

name = "./results/losses_"+timestamp+".pickle"
with open(name, 'wb') as f:
    pickle.dump(train_losses, f)

In [ ]:
name = "./results/val_losses_"+timestamp+".pickle"
with open(name, 'wb') as f:
    pickle.dump(val_losses, f)